# Importing Horizons from DUG
This notebook contains a workflow for importing horizons that were exported from DUG Insight.
DUG exports picks for all the lines in a volume for each horizon
It combines them all into a single dataframe, providing the depth to each horizon (in m, based on the selected DUG velocity field), at each common mid point (CMP) location. Each CMP also has an X and Y position (in m) that is used to calculate the distance along the line of each CMP.

The user selects which seismic line is of interest to them, and the distance along the line, along with depth to each of the exported horizons is exported to a file for use with a different notebook in this workflow.

There are a couple of bits of code that require user input for the scipt to work as designed.
These are marked accordingly.

# Table of Contents
[Importing Packages](#imports)


[Building Horizons Dataframe](#horizons)

## Importing from all the required packages
<a id="imports"></a>

In [1]:
from __future__ import print_function

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

import cartopy.feature as cfeature
import cartopy.crs as ccrs
from pyproj import Proj, transform
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from fatiando import utils, mesher, inversion
from fatiando.gravmag import talwani

import numpy as np
import pandas as pd
import math
import os
import sys
import pickle

import shapely
from shapely.geometry import LineString, mapping
from shapely.ops import polygonize
from pprint import pprint

%matplotlib inline

C:\Users\u19955\AppData\Local\Continuum\Anaconda3\envs\maps\lib\site-packages\fatiando\vis\mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


## User Action Required 1
Get all the files in the directory ready for import by:
- put all the horizon export files into a single directory
- removing any special characters in the file names (eg '%')
- name the horizon file something obvious and logical, as these names will continue throughout

<a id="horizons"></a>
## User Action Required 2
Change the path variable below to the correct directory for where your files are saved
## User Action Required 3
Ensure that the list variable called "dug_cols" contain the correct column names and in the correct order for the horizons as you exported them


In [2]:
# build a list of all the horizon files (.dat) in the specified directory
# iterate through that list, importing each file as a DataFrame
# that is appended to a list of dataframes

path = 'FromDUG/'
extn = '.dat'
dug_cols =['survey', 'line_name','cmp','x','y','twt','z']
datFiles = []
horizondataframes = []

allFiles = os.listdir(path)
for f in allFiles:
    if f[-4::] == extn:
        datFiles.append(f.split('.')[0])

for f in datFiles:
    fullPath = path + f + extn
    horizondataframes.append(pd.read_csv(fullPath, delim_whitespace=True, names=dug_cols))

In [3]:
# combine all the data into a single dataframe
horizons = pd.concat(horizondataframes)
# drop columns not need at all or yet
horizons.drop(['survey','twt', 'z'], axis =1, inplace=True)
# remove duplicates (any CMP that had more that one horizon picked on it will be duplicated)
horizons.drop_duplicates(keep='first', inplace=True)

# clean up the line names into a nicer format
def tidylinename(row):
    result = row['line_name'].replace('-','_').replace('_cable1_gun1','')
    return result

horizons['line_name'] = horizons.apply(lambda row: tidylinename(row), axis=1)

# build a unique id for each CMP
def uniqueID(row):
    result = row['line_name'] + str(row['cmp'])
    return result

horizons['uid'] = horizons.apply(lambda row: uniqueID(row), axis=1)

# set unique id as index for table
horizons.set_index('uid',drop=False, inplace=True)

# iterate through the file name array as a means of naming a new column after the horizon
# for each dataframe in array, fix line name, create uniqueID as index and join onto horizons
# via index, filling the column with the depth data for that horizon
for i in range(len(datFiles)):
    horizondataframes[i]['line_name'] = horizondataframes[i].apply(lambda row: tidylinename(row), axis=1)
    horizondataframes[i]['uid'] = horizondataframes[i].apply(lambda row: uniqueID(row), axis=1)
    horizondataframes[i].set_index('uid',drop=False, inplace=True) 
    
    new_col_name = datFiles[i] + '_z'
    horizons = pd.concat([horizons, horizondataframes[i]['z'].rename(new_col_name)],
                                       axis=1, join_axes=[horizons.index])
horizons

,line_name,cmp,x,y,uid,EocUC_z,OligUC_z,Top_EarlySag_z,Top_PreRift_z,Top_SynRift1_z,Top_SynRift2_z,Top_Volcanics_z,Waterbottom_z
uid,,,,,,,,,,,,,
BV2A1404,BV2A,1404,667330.87,6903043.06,BV2A1404,2610.2424,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV2A1408,BV2A,1408,667342.34,6903048.10,BV2A1408,2609.5833,NaN,2810.9458,NaN,NaN,NaN,NaN,NaN
BV2A1412,BV2A,1412,667353.81,6903053.14,BV2A1412,2608.9175,NaN,2811.8894,NaN,NaN,NaN,NaN,NaN
BV2A1416,BV2A,1416,667365.29,6903058.17,BV2A1416,2608.2670,NaN,2812.8364,NaN,NaN,NaN,NaN,NaN
BV2A1420,BV2A,1420,667376.76,6903063.21,BV2A1420,2607.6213,NaN,2813.7854,NaN,NaN,NaN,NaN,NaN
BV2A1424,BV2A,1424,667388.24,6903068.25,BV2A1424,2606.9722,NaN,2814.7388,NaN,NaN,NaN,NaN,1955.2505
BV2A1428,BV2A,1428,667399.71,6903073.28,BV2A1428,2606.3386,NaN,2815.6948,NaN,NaN,NaN,NaN,1955.7786
BV2A1432,BV2A,1432,667411.19,6903078.32,BV2A1432,2605.7092,NaN,2816.6530,NaN,NaN,NaN,NaN,1956.3787
BV2A1436,BV2A,1436,667422.66,6903083.36,BV2A1436,2605.0735,NaN,2817.6135,NaN,NaN,NaN,NaN,1956.9624


In [4]:
def makeLong(row):
    x = row['x']
    y = row['y']
    inProj = Proj("+proj=utm +zone=57J, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    outProj = Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    latlong = transform(inProj,outProj,x,y)
    return latlong[0]
horizons['longitude'] = horizons.apply(lambda row: makeLong(row), axis=1)

def makeLat(row):
    x = row['x']
    y = row['y']
    inProj = Proj("+proj=utm +zone=57J, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    outProj = Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    latlong = transform(inProj,outProj,x,y)
    return latlong[1]
horizons['latitude'] = horizons.apply(lambda row: makeLat(row), axis=1)

# dropping rows with no x, y coodinates
nans = horizons['x'].isnull() | horizons['y'].isnull()
horizons.drop(horizons[nans].index, inplace=True)

# cleaning up the line names into a nicer format
def tidylinename(row):
    result = row['line_name'].replace('-','_').replace('_cable1_gun1','')
    return result

horizons['line_name'] = horizons.apply(lambda row: tidylinename(row), axis=1)

In [5]:
# import ocean bottom seismometer refraction data
refract_cols = ['obs_num', 'long','lat','length_along_line','waterdepth','moho_depth']
refract = pd.read_csv('OriginalData/pts_KR16-05_Gallais_OBS_moho_20170330.dat',
                      delim_whitespace=True, names=refract_cols)

# use the OBS data to interpolate the moho depth along the seismic reflection lines
# assume that the crust thickness is constant along strike, so project along
# line of longitude for the interpolation
def interpolateMoho(row):
    if row['longitude'] > refract['long'].max():
        return 23000.0
    leftOBS = refract[refract['long'] <= row['longitude']].iloc[-1]
    rightOBS = refract[refract['long'] >= row['longitude']].iloc[0]
    longSep = rightOBS['long'] - leftOBS['long']
    longRatio = (row['longitude'] - leftOBS['long'])/longSep
    mohoSep = (rightOBS['moho_depth'] - leftOBS['moho_depth']) * 1000
    return ((leftOBS['moho_depth'] * 1000) + (longRatio * mohoSep))
horizons['moho_z'] = horizons.apply(lambda row: interpolateMoho(row), axis=1)

# adding horizons not available from DUG
# these can be commented out if not required
horizons['watersurface'] = 0.0
# horizons['moho_z'] = 23000.0
horizons['intra_mantle_z'] = 30000.0

horizons

,line_name,cmp,x,y,uid,EocUC_z,OligUC_z,Top_EarlySag_z,Top_PreRift_z,Top_SynRift1_z,Top_SynRift2_z,Top_Volcanics_z,Waterbottom_z,longitude,latitude,moho_z,watersurface,intra_mantle_z
uid,,,,,,,,,,,,,,,,,,
BV2A1404,BV2A,1404,667330.87,6903043.06,BV2A1404,2610.2424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.701516,-27.987257,21429.217386,0.0,30000.0
BV2A1408,BV2A,1408,667342.34,6903048.10,BV2A1408,2609.5833,NaN,2810.9458,NaN,NaN,NaN,NaN,NaN,160.701632,-27.987210,21429.090090,0.0,30000.0
BV2A1412,BV2A,1412,667353.81,6903053.14,BV2A1412,2608.9175,NaN,2811.8894,NaN,NaN,NaN,NaN,NaN,160.701748,-27.987163,21428.962794,0.0,30000.0
BV2A1416,BV2A,1416,667365.29,6903058.17,BV2A1416,2608.2670,NaN,2812.8364,NaN,NaN,NaN,NaN,NaN,160.701864,-27.987116,21428.835385,0.0,30000.0
BV2A1420,BV2A,1420,667376.76,6903063.21,BV2A1420,2607.6213,NaN,2813.7854,NaN,NaN,NaN,NaN,NaN,160.701979,-27.987069,21428.708090,0.0,30000.0
BV2A1424,BV2A,1424,667388.24,6903068.25,BV2A1424,2606.9722,NaN,2814.7388,NaN,NaN,NaN,NaN,1955.2505,160.702095,-27.987022,21428.580683,0.0,30000.0
BV2A1428,BV2A,1428,667399.71,6903073.28,BV2A1428,2606.3386,NaN,2815.6948,NaN,NaN,NaN,NaN,1955.7786,160.702211,-27.986976,21428.453386,0.0,30000.0
BV2A1432,BV2A,1432,667411.19,6903078.32,BV2A1432,2605.7092,NaN,2816.6530,NaN,NaN,NaN,NaN,1956.3787,160.702327,-27.986929,21428.325979,0.0,30000.0
BV2A1436,BV2A,1436,667422.66,6903083.36,BV2A1436,2605.0735,NaN,2817.6135,NaN,NaN,NaN,NaN,1956.9624,160.702443,-27.986882,21428.198684,0.0,30000.0


## User Action Required 4
- Ensure the columns are in the correct order:
- Each horizon MUST be in adjacent columns, with the top (shallowest) horizon first (eg watersurface > ocean bottom > top syn-rift > top crust > moho)
- The cell immediately below will list the columns in their current order
- The cell below that gives you a place to copy and paste the column names around into the required order

In [6]:
# listing current column order
list(horizons.columns)

['line_name',
 'cmp',
 'x',
 'y',
 'uid',
 'EocUC_z',
 'OligUC_z',
 'Top_EarlySag_z',
 'Top_PreRift_z',
 'Top_SynRift1_z',
 'Top_SynRift2_z',
 'Top_Volcanics_z',
 'Waterbottom_z',
 'longitude',
 'latitude',
 'moho_z',
 'watersurface',
 'intra_mantle_z']

In [7]:
# enter the new column order as variable new_cols
# the leading 'u' before the column name should be included

new_cols = [u'line_name',
            u'cmp',
            u'uid',
            u'x',
            u'y',
            u'longitude',
            u'latitude',
            u'watersurface',
            u'Waterbottom_z',
            u'OligUC_z',
            u'EocUC_z',            
            u'Top_EarlySag_z',
            u'Top_SynRift2_z',
            u'Top_SynRift1_z',
            u'Top_PreRift_z',
            u'moho_z',
            u'intra_mantle_z']
# changing the column order
horizons = horizons[new_cols].copy()
horizons

,line_name,cmp,uid,x,y,longitude,latitude,watersurface,Waterbottom_z,OligUC_z,EocUC_z,Top_EarlySag_z,Top_SynRift2_z,Top_SynRift1_z,Top_PreRift_z,moho_z,intra_mantle_z
uid,,,,,,,,,,,,,,,,,
BV2A1404,BV2A,1404,BV2A1404,667330.87,6903043.06,160.701516,-27.987257,0.0,NaN,NaN,2610.2424,NaN,NaN,NaN,NaN,21429.217386,30000.0
BV2A1408,BV2A,1408,BV2A1408,667342.34,6903048.10,160.701632,-27.987210,0.0,NaN,NaN,2609.5833,2810.9458,NaN,NaN,NaN,21429.090090,30000.0
BV2A1412,BV2A,1412,BV2A1412,667353.81,6903053.14,160.701748,-27.987163,0.0,NaN,NaN,2608.9175,2811.8894,NaN,NaN,NaN,21428.962794,30000.0
BV2A1416,BV2A,1416,BV2A1416,667365.29,6903058.17,160.701864,-27.987116,0.0,NaN,NaN,2608.2670,2812.8364,NaN,NaN,NaN,21428.835385,30000.0
BV2A1420,BV2A,1420,BV2A1420,667376.76,6903063.21,160.701979,-27.987069,0.0,NaN,NaN,2607.6213,2813.7854,NaN,NaN,NaN,21428.708090,30000.0
BV2A1424,BV2A,1424,BV2A1424,667388.24,6903068.25,160.702095,-27.987022,0.0,1955.2505,NaN,2606.9722,2814.7388,NaN,NaN,NaN,21428.580683,30000.0
BV2A1428,BV2A,1428,BV2A1428,667399.71,6903073.28,160.702211,-27.986976,0.0,1955.7786,NaN,2606.3386,2815.6948,NaN,NaN,NaN,21428.453386,30000.0
BV2A1432,BV2A,1432,BV2A1432,667411.19,6903078.32,160.702327,-27.986929,0.0,1956.3787,NaN,2605.7092,2816.6530,NaN,NaN,NaN,21428.325979,30000.0
BV2A1436,BV2A,1436,BV2A1436,667422.66,6903083.36,160.702443,-27.986882,0.0,1956.9624,NaN,2605.0735,2817.6135,NaN,NaN,NaN,21428.198684,30000.0


## User Action Required 5
The next cell requests user input.
Copy and paste the desired seismic line from the list provided into the text box.

In [9]:
# import the minute sampled gravity readings for each line
gravReadings = pickle.load(open('GravPerLine.pkl','rb'))
gravReadings

,datetime,line_name,latitude_dd,longitude_dd,utm_zone,x,y,length_along_line_m,water_depth_m,ground_speed_knts,heading_degT,raw_gravity_mgals,raw_acx_mgals,raw_acy_mgals,raw_eoetvoes_mgals,raw_free_air_anom_mgals,raw_bouguer_mgals,ship_eoetvoes_correction_mgals,drift_correction_mgals,corrected_abs_grav_mgals
0,2016-04-08 21:31:00,D3A_Line2,-27.318847,161.530770,57J,750426.232536,6.975709e+06,0.000000,1462.0,4.3,126.4,-2029.20,-0.0049,0.0030,-2003.7,-458.9,-458.9,25.50,-0.656791,979149.370309
1,2016-04-08 21:32:00,D3A_Line2,-27.319405,161.531962,57J,750542.939955,6.975645e+06,133.234804,1462.0,4.2,111.2,-2029.22,0.0000,-0.0027,-2004.1,-459.4,-459.4,25.12,-0.656838,979148.970262
2,2016-04-08 21:33:00,D3A_Line2,-27.319955,161.533195,57J,750663.789532,6.975582e+06,269.707244,1460.0,4.4,117.8,-2029.64,0.0009,0.0020,-2004.1,-459.4,-459.5,25.54,-0.656885,979148.970215
3,2016-04-08 21:34:00,D3A_Line2,-27.320583,161.534475,57J,750789.081051,6.975509e+06,414.276741,1459.0,4.8,113.3,-2030.48,0.0020,-0.0012,-2004.0,-459.4,-459.4,26.48,-0.656932,979149.070168
4,2016-04-08 21:35:00,D3A_Line2,-27.321235,161.535688,57J,750907.719403,6.975435e+06,554.277041,1458.0,4.7,119.0,-2031.16,-0.0005,-0.0026,-2004.3,-459.7,-459.7,26.86,-0.656979,979148.770121
5,2016-04-08 21:36:00,D3A_Line2,-27.321943,161.536962,57J,751032.167920,6.975354e+06,702.595512,1459.0,4.9,114.7,-2031.12,0.0007,-0.0013,-2004.4,-459.9,-459.9,26.72,-0.657026,979148.670074
6,2016-04-08 21:37:00,D3A_Line2,-27.322592,161.538220,57J,751155.265361,6.975279e+06,846.413415,1460.0,4.5,121.5,-2031.63,-0.0017,-0.0016,-2004.9,-460.4,-460.4,26.73,-0.657072,979148.170028
7,2016-04-08 21:38:00,D3A_Line2,-27.323225,161.539517,57J,751282.189717,6.975207e+06,992.726094,1462.0,4.5,121.7,-2032.00,-0.0020,0.0031,-2004.7,-460.3,-460.3,27.30,-0.657119,979148.369981
8,2016-04-08 21:39:00,D3A_Line2,-27.323855,161.540815,57J,751409.285156,6.975134e+06,1139.004562,1463.0,4.9,122.2,-2032.50,-0.0009,-0.0006,-2005.0,-460.6,-460.6,27.50,-0.657166,979148.069934
9,2016-04-08 21:40:00,D3A_Line2,-27.324513,161.542113,57J,751536.315188,6.975059e+06,1286.815123,1465.0,4.5,118.2,-2033.34,0.0031,-0.0017,-2005.7,-461.3,-461.3,27.64,-0.657213,979147.369887


In [8]:
# import the minute sampled gravity readings for each line
gravReadings = pickle.load(open('GravPerLine.pkl','rb'))

# build list of interpreted seismic lines that don't have gravity readings
allLineNames = horizons['line_name'].unique()
notComparableLines = []
for sline in allLineNames:
    if sline in gravReadings.keys():
        continue
    else:
        notComparableLines.append(sline)

# list the available seismic lines
allLineNames = horizons.line_name.unique()
print('The available lines are', allLineNames)

# ask user which line they would like the horizons extracted for
# if they chose one without seismic data, get them to chose again
# done like this to make the user aware that there isn't perfect alingment between
# seismic available in DUG and gravity from survey
while True:
    lineChoice = raw_input('Which line would you like to generate polygons for?\n' +
                   'The name needs to exactly match the list above excluding quote marks. ')
    if lineChoice in notComparableLines:
        print('Selected line has no gravity data, please choose again')
    else:
        break
# lineChoice = 'D3A_Line2'

# subsetting the requested data
linePoints = horizons['line_name'] == lineChoice
linePoints = horizons[linePoints].copy()
gravData = gravReadings[lineChoice]

The available lines are ['BV2A' 'D1B_Line10' 'D1B_Line11' 'D1B_Line12' 'D1B_Line13' 'D1B_Line8'
 'D2A_Line2' 'D3A_Line1' 'D3A_Line2' 'D3A_Line3' 'D3A_Line4' 'D3A_Line5'
 'D3A_Line6' 'D3A_Line7' 'D3A_Line8' 'EWmcs' 'D1B_Line9m' 'BB1B']
Which line would you like to generate polygons for?
The name needs to exactly match the list above excluding quote marks. D3A_Line2


In [9]:
# just to check that all the horizons are correctly sorted by depth
# col_num is the first column with depth data, ie watersurface
def integrityCheck(row, col_num):
    for i in range(col_num, len(row)-1):   # len(row) - 1 because "check" column is created by now
        if np.isnan(row[i]):
            continue
        for j in range(i,len(row)-1):
            if np.isnan(row[j]):
                continue
            if row[i] > row[j]:
                return False
    return True

start_col = linePoints.columns.get_loc('watersurface')
linePoints['check'] = linePoints.apply(lambda row: integrityCheck(row, start_col), axis=1)

print('This code cell should present an empty dataframe.')
print('If there is any data in it then that means that the columns are not in the correct order,\n')
print('ie there is an older horizon above a younger one for a given CMP.\n')
print(linePoints[~linePoints['check']])

# removing the checking column
linePoints.drop('check', axis=1, inplace=True)

This code cell should present an empty dataframe.
If there is any data in it then that means that the columns are not in the correct order,

ie there is an older horizon above a younger one for a given CMP.

Empty DataFrame
Columns: [line_name, cmp, uid, x, y, longitude, latitude, watersurface, Waterbottom_z, OligUC_z, EocUC_z, Top_EarlySag_z, Top_SynRift2_z, Top_SynRift1_z, Top_PreRift_z, moho_z, intra_mantle_z, check]
Index: []


In [11]:
linePoints

,line_name,cmp,uid,x,y,longitude,latitude,watersurface,Waterbottom_z,OligUC_z,EocUC_z,Top_EarlySag_z,Top_SynRift2_z,Top_SynRift1_z,Top_PreRift_z,moho_z,intra_mantle_z
uid,,,,,,,,,,,,,,,,,
D3A_Line23168,D3A_Line2,3168,D3A_Line23168,753911.52,6973682.55,161.566385,-27.336485,0.0,1480.5315,1885.5814,1896.9099,1937.1719,NaN,NaN,2116.1614,23685.491715,30000.0
D3A_Line23172,D3A_Line2,3172,D3A_Line23172,753922.33,6973676.27,161.566495,-27.336540,0.0,1480.6348,1884.4781,1897.5381,1937.6407,NaN,NaN,2118.4670,23685.714555,30000.0
D3A_Line23176,D3A_Line2,3176,D3A_Line23176,753933.14,6973669.98,161.566605,-27.336594,0.0,1480.7467,1884.0559,1898.1674,1938.0994,NaN,NaN,2120.7720,23685.937399,30000.0
D3A_Line23180,D3A_Line2,3180,D3A_Line23180,753943.95,6973663.70,161.566716,-27.336649,0.0,1480.7803,1884.0642,1898.7987,1938.5673,NaN,NaN,2122.0900,23686.160239,30000.0
D3A_Line23184,D3A_Line2,3184,D3A_Line23184,753954.76,6973657.42,161.566826,-27.336704,0.0,1480.8221,1884.4595,1899.4202,1939.0248,NaN,NaN,2122.6655,23686.383079,30000.0
D3A_Line23188,D3A_Line2,3188,D3A_Line23188,753965.57,6973651.14,161.566937,-27.336758,0.0,1480.8877,1885.1220,1900.0543,1939.4827,NaN,NaN,2123.2393,23686.605919,30000.0
D3A_Line23192,D3A_Line2,3192,D3A_Line23192,753976.38,6973644.86,161.567047,-27.336813,0.0,1480.9770,1886.3265,1900.6900,1939.9520,NaN,NaN,2123.8123,23686.828760,30000.0
D3A_Line23196,D3A_Line2,3196,D3A_Line23196,753987.19,6973638.57,161.567158,-27.336868,0.0,1481.0277,1886.9912,1901.3270,1940.4100,NaN,NaN,2124.3843,23687.051605,30000.0
D3A_Line23200,D3A_Line2,3200,D3A_Line23200,753998.00,6973632.29,161.567268,-27.336922,0.0,1481.0475,1887.2388,1901.9651,1940.8798,NaN,NaN,2124.9546,23687.274446,30000.0


In [12]:
# sort the data points, to ensure they are in in one line from the first reading to the last
# assume that the data are:
# # in a straight line
# # not directly north-south (ie no repeating X/long coordinates)
# # the coordinate system from DUG doesn't reset mid line (ie don't cross UTM zones)
linePoints.sort_values('x', axis=0, inplace=True)

# clean up CMPs where fold is decreasing and data is less good
# by simply removing this much from the start and end of the seismic line
# based on visual estimation of data quality decrease from DUG
# NOTE: the export only sends out every 4th CMP, so need to divide the number by 4
badCMP = 250
rowsToCut = int(250/4)
linePoints = linePoints.iloc[rowsToCut:-rowsToCut].copy()

# reindex the dataframe as need incremental index numbers for next operations
linePoints.reset_index(drop=True, inplace=True)

# taking the CMP location at the zero-point (based on sort order)
x0 = linePoints['x'].iloc[0]
y0 = linePoints['y'].iloc[0]

# calculating the distance from the zero-point of each subsequent CMP
def calc_distance(row):
    return math.sqrt((row['x'] - x0) ** 2 + (row['y'] - y0) ** 2)

linePoints['length_along_line'] = linePoints.apply(lambda row: calc_distance(row), axis = 1)

# calculating a horizon to be assigned as potential pre-rift sediment
# based on the assumption that the syn-rift faults were also the pre-rift faults
# so the areas with thicker syn-rift packages become the areas with thickest pre-rift sediment.
# calculated based on the depth of the CMP in question as a proportion of the maximum variation in depth
# between the highest point of the base syn-rift, and the lowest  x preRiftSedDepth variable 
preriftdeep = linePoints['Top_PreRift_z'].max()
preriftshallow = linePoints['Top_PreRift_z'].min()
preriftrange = preriftdeep - preriftshallow
preRiftSedDepth = 0
if preRiftSedDepth != 0:
    new_col_index = linePoints.columns.get_loc('Top_PreRift_z') + 1
    if 'Top_Basement_z' in list(linePoints.columns):
        linePoints.drop('Top_Basement_z', inplace = True, axis = 1)
    linePoints.insert(new_col_index, 'Top_Basement_z', linePoints['Top_PreRift_z']  + preRiftSedDepth * (linePoints['Top_PreRift_z'] - preriftshallow)/preriftrange)

# add start and end points beyond beyond the edge of the line
def extendCrossSection(df, line_extn_dist):
    west_extn = df.iloc[0].copy()
    west_extn['length_along_line'] = - line_extn_dist
    west_extn = pd.DataFrame(west_extn).transpose()
    east_extn = df.iloc[-1].copy()
    east_extn['length_along_line'] = east_extn['length_along_line'] + line_extn_dist
    east_extn = pd.DataFrame(east_extn).transpose()
    df = pd.concat([west_extn,df,east_extn], axis=0, ignore_index=True)
    return df
    
linePoints = extendCrossSection(linePoints, 300000)

# build a list of all columns that contain horizon data
cols = linePoints.columns
horCols = []
for col_name in cols:
    if col_name == 'watersurface' or col_name[-2:] =='_z':
        if linePoints[col_name].isnull().all():
            continue
        horCols.append(col_name)

depthscalefactor = 1
if depthscalefactor != 1:    
    for col in horCols:
        linePoints[col] = linePoints[col] * depthscalefactor
    
linePoints

,line_name,cmp,uid,x,y,longitude,latitude,watersurface,Waterbottom_z,OligUC_z,EocUC_z,Top_EarlySag_z,Top_SynRift2_z,Top_SynRift1_z,Top_PreRift_z,moho_z,intra_mantle_z,length_along_line
0,D3A_Line2,1832,D3A_Line21832,750308,6.97579e+06,161.53,-27.3181,0,1465.58,NaN,NaN,NaN,NaN,NaN,1788.64,23611.2,30000,-300000
1,D3A_Line2,1832,D3A_Line21832,750308,6.97579e+06,161.53,-27.3181,0,1465.58,NaN,NaN,NaN,NaN,NaN,1788.64,23611.2,30000,0
2,D3A_Line2,1836,D3A_Line21836,750319,6.97579e+06,161.53,-27.3182,0,1465.68,NaN,NaN,NaN,NaN,NaN,1787.13,23611.4,30000,12.5387
3,D3A_Line2,1840,D3A_Line21840,750330,6.97578e+06,161.53,-27.3182,0,1465.96,NaN,NaN,NaN,NaN,NaN,1785.55,23611.7,30000,25.0774
4,D3A_Line2,1844,D3A_Line21844,750340,6.97577e+06,161.53,-27.3183,0,1465.96,NaN,NaN,NaN,NaN,NaN,1784.12,23611.9,30000,37.6161
5,D3A_Line2,1848,D3A_Line21848,750351,6.97577e+06,161.53,-27.3183,0,1465.94,NaN,NaN,NaN,NaN,NaN,1782.69,23612.1,30000,50.1462
6,D3A_Line2,1852,D3A_Line21852,750362,6.97576e+06,161.53,-27.3184,0,1465.94,NaN,NaN,NaN,NaN,NaN,1781.44,23612.3,30000,62.6799
7,D3A_Line2,1856,D3A_Line21856,750373,6.97576e+06,161.53,-27.3184,0,1465.9,NaN,NaN,NaN,NaN,NaN,1781.48,23612.6,30000,75.2186
8,D3A_Line2,1860,D3A_Line21860,750384,6.97575e+06,161.53,-27.3185,0,1465.87,NaN,NaN,NaN,NaN,NaN,1782.67,23612.8,30000,87.7574
9,D3A_Line2,1864,D3A_Line21864,750395,6.97574e+06,161.53,-27.3186,0,1465.83,NaN,NaN,NaN,NaN,NaN,1783.41,23613,30000,100.296


In [13]:
def findIntersection(col_name, start_position, direction, extrap_dist):   
    """
    this function linearly extends the specific horizon (column name) in the 
    specified direction ('left' or 'right') until it intersects another horizon.
    The extrapolation is done in a straight line, defined by the continuation of
    a line between the last picked point to the depth value of the horizon several
    rows back. The number of rows is provided as the extrap_dist parameter.
    The start_position parameter is the last known value, and the point that the line
    will be extrapolated from
    """    
    col_num = linePoints.columns.get_loc(col_name)
    start_value = linePoints.loc[start_position,col_name]
    intersection_found = False    
    if direction == 'left':
        step = -1
        other_value = linePoints.loc[start_position + (extrap_dist * - step),col_name]  
        limit = linePoints.index[0]       
        slope = (other_value - start_value) / extrap_dist
    else:
        step = 1
        other_value = linePoints.loc[start_position + (extrap_dist * - step),col_name]  
        limit = linePoints.index[-1]
        slope = (start_value - other_value) / extrap_dist
      
    # for each CMP beyond the edge of the current line
    for point in range(start_position, limit, step):
        next_point = point + step
        # update the next CMP along with the extrapolated value        
        linePoints.loc[next_point,col_name] = linePoints.loc[point,col_name] + (step * slope)
        # check if the horizon has crossed any of the others
        for col in horCols:
            if linePoints.columns.get_loc(col) == col_num:
                continue
            if linePoints.columns.get_loc(col) < col_num:
                if linePoints.loc[next_point,col] >= linePoints.loc[next_point,col_name]:
                    # if a shallower horizon has a depth >= extrpolated value, assign that depth
                    # to the horizon in question, record the details of the intersection
                    linePoints.loc[next_point, col_name] = linePoints.loc[next_point,col]
                    intersection_found = True
                    break
            if linePoints.columns.get_loc(col) > col_num:
                if linePoints.loc[next_point,col] <= linePoints.loc[next_point,col_name]:
                    # if a deeper horizon has a depth <= extrpolated value, assign that depth
                    # to the horizon in question, record the details of the intersection
                    linePoints.loc[next_point, col_name] = linePoints.loc[next_point,col]
                    intersection_found = True
                    break           
        if intersection_found:
            break

# figure out which columns need extending        
fullWidthHorizons = []
shortHorizons = []
for col_name in horCols:
    if (~linePoints[col_name].isnull()).all():
        fullWidthHorizons.append(col_name)
    else:
        shortHorizons.append(col_name)

# extend short horizons and fill in the gaps in horizons
for col_name in shortHorizons:
    col_num = linePoints.columns.get_loc(col_name)        
    watersurface = linePoints.columns.get_loc('watersurface') # column number of water surface
    
    notBlankFlags = ~linePoints[col_name].isnull()   # if cell is not NaN
    first_value = linePoints[notBlankFlags].index[0] # first non-NaN location
    last_value = linePoints[notBlankFlags].index[-1] # last non-NaN loction
    startFlags = linePoints.index >= first_value     # all the cells after the first non-NaN
    endFlags = linePoints.index <= last_value        # all the cells before the last non-NaN
    colRange = startFlags & endFlags                 # all the cells between the first and last non-NaN
   
    # dealing with edge cases for all short horizons
    findIntersection(col_name, first_value, 'left', 10)
    findIntersection(col_name, last_value, 'right', 10)
    gaps = []
    lineSegs = []
    # for short, non-broken horizons, add lines coords to list
           
    notBlankFlags = ~linePoints[col_name].isnull()   # if cell is not NaN
    first_value = linePoints[notBlankFlags].index[0] # first non-NaN location
    last_value = linePoints[notBlankFlags].index[-1] # last non-NaN loction
    startFlags = linePoints.index >= first_value     # all the cells after the first non-NaN
    endFlags = linePoints.index <= last_value        # all the cells before the last non-NaN
    colRange = startFlags & endFlags                 # all the cells between the first and last non Nan
            
    # for cases with breaks in horizons
    if not (colRange == notBlankFlags).all():
        gapPoints = ~(colRange == notBlankFlags)
        midLineGaps = linePoints[gapPoints].copy() # the cells that are gaps
        for i in list(midLineGaps.index):
            # for the start and end of each gap, record location in gaps list
            if (i-1) not in list(midLineGaps.index):
                start = i
            if (i+1) not in list(midLineGaps.index):
                end = i        
                gaps.append([start,end])

    for gap in gaps:
        start = gap[0]
        end = gap[1]
        left_edge = linePoints.loc[start - 1,col_name]       # depth value at last point left of gap
        right_edge = linePoints.loc[end + 1, col_name]       # depth value at last point right of gap
        slope = (right_edge - left_edge) / (end - start + 2) # slope across gap
        midLineGaps.loc[start,col_name] = linePoints.loc[start - 1, col_name] + slope

        for i in range(start+1, end + 1, 1):
            # linearly interpolated gap filling
            midLineGaps.loc[i,col_name] = midLineGaps.loc[i - 1, col_name] + slope
        # if midLineGaps fails integrity check (ie another horizon crosses the one in question)
        if len(midLineGaps[~midLineGaps.apply(lambda row: integrityCheck(row, watersurface), axis = 1)]) > 0:
            # extend horizons into gap to intersecting horizon
            findIntersection(col_name, start-1, 'right', 10)
            findIntersection(col_name, end+1, 'left', 10)
        else:
            # fill the horizon with a linear interpolation
            for i in range(start, end + 1, 1):
                linePoints.loc[i,col_name] = linePoints.loc[i - 1, col_name] + slope

# find all intersections and line ends
intersections = []
segments = []
segmentNames = []
Xmin = linePoints['length_along_line'].min()
Xmax = linePoints['length_along_line'].max()

# for each column, iterate through and find intersections
for col_name in horCols:
    col_num = linePoints.columns.get_loc(col_name)
    first_col = linePoints.columns.get_loc(horCols[0])
    last_col = linePoints.columns.get_loc(horCols[-1])
    segment = []
    # for each row in each column
    for row in linePoints.index:    
        isSegmentEnd = False
        x = linePoints.loc[row,'length_along_line']
        z = linePoints.loc[row,col_name]        
        # if no value in cell, go to next row
        if np.isnan(z):
            continue           
        # if row is left edge of cross-section, start new segment, record as intersection
        if x == Xmin:
            intersections.append((x, z))               
        # if row is left edge of cross section, record as intersection, flag as segment end
        if x == Xmax:
            intersections.append((x, z))
            isSegmentEnd = True
        for lower_col_num in range(first_col,last_col + 1):
            if lower_col_num == col_num:
                continue
            lower_col_name = linePoints.columns[lower_col_num]
            if (not np.isnan(z) and 
                not np.isnan(linePoints.loc[row,lower_col_name]) and 
                z == linePoints.loc[row,lower_col_name]):
                intersections.append((x, z))
                isSegmentEnd = True
                break
        if len(segment) == 0:
            isSegmentEnd = False
        if isSegmentEnd:
            segment.append((x, z))
            segments.append(segment)
            segmentNames.append(col_name + str(row))
            segment = []
            if row + 1 in linePoints.index and not np.isnan(linePoints.loc[row+1,col_name]):
                segment.append((x, z))
        if not isSegmentEnd:
            segment.append((x, z))    

# building left vertical segments
for col_name in horCols:
    segment = []
    col_num = linePoints.columns.get_loc(col_name)
    LAL = linePoints.columns.get_loc('length_along_line')
    end_col_num = linePoints.columns.get_loc(horCols[-1])
    x = linePoints.iloc[0,LAL]
    z = linePoints.iloc[0,col_num]    
    if col_name == horCols[-1]:
        break
    if np.isnan(z):
        continue
    segment.append((x, z))
    for next_col_num in range(col_num + 1,end_col_num + 1):
        z1 = linePoints.iloc[0,next_col_num]
        if np.isnan(z1):
            continue
        segment.append((x, z1))
        break
    segments.append(segment)
    segmentNames.append('left_vert_' + col_name + '_down')

# building right vertical segments
for col_name in horCols:
    segment = []
    col_num = linePoints.columns.get_loc(col_name)
    LAL = linePoints.columns.get_loc('length_along_line')
    end_col_num = linePoints.columns.get_loc(horCols[-1])
    x = linePoints.iloc[-1,LAL]
    z = linePoints.iloc[-1,col_num]    
    if col_name == horCols[-1]:
        break
    if np.isnan(z):
        continue
    segment.append((x, z))
    for next_col_num in range(col_num + 1,end_col_num + 1):
        z1 = linePoints.iloc[-1,next_col_num]
        if np.isnan(z1):
            continue
        segment.append((x, z1))
        break
    segments.append(segment)
    segmentNames.append('right_vert_' + col_name + '_down')

# len(segments)
# segmentNames
# for i in range(len(segments)):
#     print(str(i)+'. ' + str(len(segments[i])) + '   '+ segmentNames[i])

shapelysegs = []
for seg in segments:
    shapelysegs.append(LineString(seg))
shapelypolys = list(polygonize(shapelysegs))
shapelycentroids = []
for poly in shapelypolys:
    shapelycentroids.append(poly.centroid)
shapelyreppoints = []
for poly in shapelypolys:
    shapelyreppoints.append(poly.representative_point())

print('You have made ' + str(len(shapelypolys)) + ' polygons')

In [ ]:
refract.columns
# plt.plot(refract['length_along_line'],refract['moho_depth'])
plt.plot(refract['long'],refract['moho_depth']*1000, 'k-')
plt.plot(refract['long'],refract['waterdepth']*1000, 'b-')
plt.plot(linePoints['longitude'],linePoints['moho_z'], 'gx')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
fig_size = [20,20] 
font = {'size':25}
# axis = []
axis = [0,35000,1000,6000]
plt.subplot(111)
# plt.axis(calc_area)
plt.axis(axis)

# may need to adjust these coordinates to ensure that all polygons and their numbers are visible
# pattern is Xmin, Xmax, Ymin, Ymax

NPpolys = []
for i in range(len(shapelypolys)):
    NPpolys.append(Polygon(list(shapelypolys[i].exterior.coords)))

p = PatchCollection(NPpolys,cmap=matplotlib.cm.jet, alpha = 1) # alpha = transparency
colours = np.arange(len(shapelypolys))
p.set_array(colours)
plt.gca().add_collection(p)    



plt.gca().invert_yaxis()
plt.xlabel('Length Along Line (m)')
plt.ylabel('Depth (m below sea level)')    

for i in range(len(shapelyreppoints)):
    if list(shapelyreppoints[i].coords)[0][0] > axis[1]:
        x = axis[1]
    elif list(shapelyreppoints[i].coords)[0][0] < axis[0]:
        x = axis[0]
    else:
        x = list(shapelyreppoints[i].coords)[0][0]
    if list(shapelyreppoints[i].coords)[0][1] > axis[3]:
        z = axis[3]
    elif list(shapelyreppoints[i].coords)[0][1] < axis[2]:
        z = axis[2]
    else:
        z = list(shapelyreppoints[i].coords)[0][1]
#     x = list(shapelyreppoints[i].coords)[0][0]
#     z = list(shapelyreppoints[i].coords)[0][1]
    plt.plot(x,z, 'ok')
    plt.text(x+100, z+100, str(i), fontdict = font)
plt.rcParams["figure.figsize"] = fig_size
# plt.savefig("variableprerift.svg")
plt.show()

# User Input Required
Assign model densities to the polygons using the numbers indicated on the plot to show you which polygon is which. The number of densities must exactly match the number of polygons, so you may need to copy and paste or delete some entries.

In [ ]:
# using the densities backed out of the stacking velocites by Petkovic et al. 2010
water = 1010.0
passivemarginsediment = 1850.0
sag = 2130.0
syn_rift = 2310.0
sedcrust = 2440.0
crystalcrust = 2670
mantle = 3300.0

densities = [{'density':water}, # 0
             {'density':passivemarginsediment}, # 1
             {'density':sag}, # 2
             {'density':sag}, # 3
             {'density':syn_rift}, # 4
             {'density':syn_rift}, # 5
             {'density':sag}, # 6
             {'density':sag}, # 7
             {'density':syn_rift}, # 8
             {'density':mantle}, # 9
             {'density':sedcrust}, # 10
             {'density':crystalcrust}, # 11
             {'density':mantle}] # 12


den_set = set()
for den in densities:
    den_set.add(den['density'])
    
# import matplotlib.patches as patches
# import matplotlib.colors as colors
# import math


# fig = plt.figure()
# ax = fig.add_subplot(111)

# ratio = 1.0 / 3.0
# count = math.ceil(math.sqrt(len(colors.cnames)))
# x_count = count * ratio
# y_count = count / ratio
# x = 0
# y = 0
# w = 1 / x_count
# h = 1 / y_count

# for c in colors.cnames:
#     pos = (x / x_count, y / y_count)
#     ax.add_patch(patches.Rectangle(pos, w, h, color=c))
# #     ax.annotate(c, xy=pos)
#     ax.text(pos[0], pos[1], c, fontdict = font)
#     if y >= y_count-1:
#         x += 1
#         y = 0
#     else:
#         y += 1

# plt.show()

In [ ]:
den_col_dict = {1010.0: 'cornflowerblue',
                 1850.0: 'khaki',
                 2130.0: 'yellow',
                 2310.0: 'gold',
                 2440.0: 'chocolate',
                 2670.0: 'hotpink',
                 3300.0: 'olive'}

# den_col_dict = {1060.0: 'cornflowerblue',
#                  2000.0: 'khaki',
# #                  2400.0: 'yellow',
#                  2650.0: 'hotpink',
#                  3300.0: 'olive'}

# den_col_dict = {}
# for den in den_set:
#     col = raw_input('Density: '+ str(den) + '. What colour would you like this density to be drawn as? ')
#     den_col_dict[den] = col

den_col_dict

In [ ]:
# pulling out the max and min values of the bounds of the data
allCoords = []
for poly in shapelypolys:
    coords = list(poly.exterior.coords)
    for point in coords:
        allCoords.append(point)

arr = np.array(allCoords)

Xmin = np.nanmin(arr[:,0])
XminDisplay = 0
line_extn = abs(Xmin - XminDisplay)
Xmax = np.nanmax(arr[:,0])
XmaxDisplay = Xmax - line_extn
Zmin = 0
ZminDisplay = 0
Zmax = np.nanmax(arr[:,1])
ZmaxDisplay = 6200

calc_area = [Xmin, Xmax, Zmin, Zmax]
draw_area = [XminDisplay, XmaxDisplay, ZminDisplay, ZmaxDisplay]

modelX = linePoints[linePoints['length_along_line'] == 0].iloc[0,linePoints.columns.get_loc('x')]
modelY = linePoints[linePoints['length_along_line'] == 0].iloc[0,linePoints.columns.get_loc('y')]
measureX = gravData.loc[0,'x']
measureY = gravData.loc[0,'y']
offset = math.sqrt((modelX - measureX)**2 + (modelY - measureY)**2)
offset

if modelX > measureX:
    offset *= -1
offset

In [ ]:
def talwaniCalc(shapes,densities, area):
    """
    Performs the gravity forward model calculation according to
    Talwani et al.'s method as implemented by Fatiando
    
    Takes an array of polygon coordinates (as an array of [X,Y] points)
    and an array of dicts of the same length, each dict having the key
    'density', and the value of the density for the corresponding polygon
    (by index number) in kgm^-3
    """
    polys = []
    for poly in shapes:
        coords = list(poly.exterior.coords[::-1])
        coords = coords[0:-1]
        polys.append(coords)

    PolyDens = []
    for i in range(len(polys)):
        PolyDens.append(mesher.Polygon(polys[i],densities[i]))
       
    xp = np.linspace(area[0], area[1], num=1000) # define measurement points and compute gravity
    zp = np.zeros_like(xp)
    gz = talwani.gz(xp, zp, PolyDens)
    model = pd.DataFrame({'xp': xp, 'gz':gz})
    return model

if len(shapelypolys) > len(densities):          # dummy checking
    print("One of your polygons is missing a density - check your inputs")
elif len(shapelypolys) < len(densities):
    print("You have more density values than polygons - check your inputs")
    
#running the gravity model
model = talwaniCalc(shapelypolys, densities, calc_area)

In [ ]:
# map of the area
ax1 = plt.subplot2grid((2,4), (0,0), colspan=2, projection=ccrs.PlateCarree())
plt.suptitle(lineChoice + ' with up to ' + str(preRiftSedDepth) + 'm of pre-rift sediment', fontsize=20)
fig_size = [20,10] 
plt.rcParams["figure.figsize"] = fig_size
matplotlib.rcParams.update({'font.size': 12})
map_extent = [150, 165, -30, -22]
xticks = np.linspace(map_extent[0],map_extent[1],6)
yticks = np.linspace(map_extent[2], map_extent[3],6)
ax1.set_extent(map_extent)
gl = ax1.gridlines(xlocs = xticks, ylocs = yticks)
gl.xlabels_bottom = True
gl.ylabels_left = True
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
ax1.coastlines('50m')
ax1.stock_img()
ax1.add_feature(cfeature.LAND)
ax1.add_feature(cfeature.COASTLINE)
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')
ax1.plot(refract['long'],refract['lat'],color='navy', linewidth=2,
         marker='.',transform=ccrs.PlateCarree())
for line in gravReadings:
    ax1.plot(gravReadings[line]['long'],gravReadings[line]['lat'],color='black', linewidth=2,
         marker='.',transform=ccrs.PlateCarree())
ax1.plot(linePoints['longitude'],linePoints['latitude'],color='red', linewidth=2,
         marker='.',transform=ccrs.PlateCarree())

# display image of seismic line
# plt.subplot2grid((2,4), (1,0), colspan=2)
# image = plt.imread('FromDUG/' + lineChoice + '.PNG')
# plt.imshow(image)
# plt.axis('off')

# draw seismic based cross-section
plt.subplot2grid((2,4), (0,2), colspan=2)
plt.axis(draw_area)
legendpatches = []
inlegend = []
for i in range(len(shapelypolys)):
    poly = Polygon(list(shapelypolys[i].exterior.coords), closed = True,
                   color = den_col_dict[densities[i]['density']])
    plt.gca().add_patch(poly)
    shallowpoint = np.array(shapelypolys[i].exterior.xy)[1].min()
    if (shallowpoint < draw_area[3]) and (densities[i]['density'] not in inlegend):
        inlegend.append(densities[i]['density'])
        legendpatches.append(mpatches.Patch(facecolor = den_col_dict[densities[i]['density']],
                                            edgecolor='black',
                                            label = str(int(densities[i]['density']))))
plt.gca().invert_yaxis()
plt.xlabel('Length Along Line (m)')
# plt.gca().get_xaxis().set_visible(False)
plt.ylabel('Depth (m below sea level)')
legtitle = 'kg/m^3'
plt.legend(handles=legendpatches, title='Density\n' + legtitle)

# draw line charts to compare model and actual gravity
plt.subplot2grid((2,4), (1,2), colspan=2)
gravReadingToUse = 'free_air_anom'
model['shifted'] = model['gz'] - model[model['xp'] >= 0].iloc[0,model.columns.get_loc('gz')] + gravData.iloc[0,gravData.columns.get_loc(gravReadingToUse)]
Xmin = draw_area[0]
Xmax = draw_area[1]
ModelViewRange = model[(Xmin <= model['xp']) & (Xmax >= model['xp'])]
modelMax = ModelViewRange['shifted'].max()
modelMin = ModelViewRange['shifted'].min()
dataViewRange = gravData[(draw_area[0] <= gravData['length_along_line']) & (draw_area[1] >= gravData['length_along_line'])]
gravMax = dataViewRange[gravReadingToUse].max()
gravMin = dataViewRange[gravReadingToUse].min()
axisEdge = (max(gravMax,modelMax) - min(gravMin,modelMin))/10
gravAxis = [min(gravMin,modelMin) - axisEdge, max(gravMax,modelMax) + axisEdge]
plt.plot(model['xp'] - offset, model['shifted'], 'r-', label ='Model Gravity')
plt.plot(gravData['length_along_line'], gravData[gravReadingToUse],'b-', label='Measured Free Air Anomaly')
plt.ylabel('(mgals)')
plt.xlabel('Length Along Line (m)')
plt.axis(draw_area)
plt.xlim(draw_area[0],draw_area[1])
plt.ylim(gravAxis)
plt.legend()
# plt.tight_layout()
plt.savefig("avelowdensity.svg")
plt.show()

In [ ]:
font = {'size':25}
matplotlib.rcParams.update({'font.size': 20})
ax1 = plt.subplot(211,  projection=ccrs.PlateCarree())
plt.gcf()
plt.suptitle(lineChoice, fontsize=30)

map_extent = [150, 165, -30, -25]
ax1.set_extent(map_extent)
gl = ax1.gridlines()
gl.xlabels_bottom = True
gl.ylabels_left = True
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
ax1.coastlines('50m')
ax1.stock_img()
ax1.add_feature(cfeature.LAND)
ax1.add_feature(cfeature.COASTLINE)
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')

for line in gravReadings.keys():
    ax1.plot(gravReadings[line]['long'],gravReadings[line]['lat'],color='black', linewidth=2,
         marker='.',transform=ccrs.PlateCarree())
ax1.plot(gravReadings[lineChoice]['long'],gravReadings[lineChoice]['lat'],color='red', linewidth=2,
         marker='.',transform=ccrs.PlateCarree())

plt.subplot(212)
plt.plot(gravReadings[lineChoice]['length_along_line'], gravReadings[lineChoice]['free_air_anom'])
plt.ylabel('Free Air Anomaly (mgals)')
plt.xlabel('Length Along Line (m)')
fig_size = [20,20] 
plt.rcParams["figure.figsize"] = fig_size
plt.tight_layout()
plt.savefig("gravData.svg")
plt.show()

In [ ]:
fig_size = [20,20] 
font = {'size':25}
# axis = [3000,5000,1600,2000]
axis = [0,30000,1000,6000]

plt.subplot(311)

plt.axis(axis)
for seg in shapelysegs:
    plt.plot(seg.xy[0],seg.xy[1])
    
plt.gca().invert_yaxis()
# plt.xlabel('Length Along Line (m)')
plt.ylabel('Depth (m below sea level)') 
plt.gca().get_xaxis().set_visible(False)
plt.subplot(312)
plt.axis(axis)
plt.rcParams["figure.figsize"] = fig_size
# may need to adjust these coordinates to ensure that all polygons and their numbers are visible
# pattern is Xmin, Xmax, Ymin, Ymax

NPpolys = []
for i in range(len(shapelypolys)):
    NPpolys.append(Polygon(list(shapelypolys[i].exterior.coords)))

p = PatchCollection(NPpolys,cmap=matplotlib.cm.jet, alpha = 1) # alpha = transparency
colours = np.arange(len(shapelypolys))
p.set_array(colours)
plt.gca().add_collection(p)    



plt.gca().invert_yaxis()
plt.gca().get_xaxis().set_visible(False)
# plt.xlabel('Length Along Line (m)')
plt.ylabel('Depth (m below sea level)')    

# for i in range(len(shapelyreppoints)):
#     if list(shapelyreppoints[i].coords)[0][0] > axis[1]:
#         x = axis[1]
#     else:
#         x = list(shapelyreppoints[i].coords)[0][0]
#     if list(shapelyreppoints[i].coords)[0][1] > axis[3]:
#         z = axis[3]
#     else:
#         z = list(shapelyreppoints[i].coords)[0][1]
#     plt.plot(x,z, 'ok')
#     plt.text(x+100, z+100, str(i), fontdict = font)

plt.subplot(313)
plt.axis([axis[0],axis[1],3245,3285])
plt.plot(model['xp'], model['gz'], 'r-', label ='Model Gravity')
plt.xlabel('Length Along Line (m)')
plt.ylabel('Modelled Gravity')  
plt.tight_layout()
plt.savefig("gravModel.svg")
plt.show()

In [ ]:
plt.plot(linePoints['x'],linePoints['y'],'kx')
plt.plot(gravData['x'], gravData['y'], 'g-')
plt.plot(modelX,modelY,'bo')
plt.plot(measureX, measureY, 'ro')
plt.xlim([750000,755000])
plt.ylim([6974000, 6976000])
plt.show()

In [ ]:
linePoints.loc[300:335]

In [ ]:
# import the minute sampled gravity readings for each line
funclinePoints = pickle.load(open('linePointsFunction.pkl','rb'))
funclinePoints.drop(['check','top_basement_z'], inplace = True, axis = 1)
funclinePoints.loc[300:335]

In [ ]:
for col in list(linePoints.columns):
    print(col, linePoints[col].equals(funclinePoints[col]))
print('\n\n')
for col in horCols:
    same = True
    for i in list(linePoints.index):
        if np.isnan(linePoints.loc[i, col]) and np.isnan(funclinePoints.loc[i,col]):
            continue
        elif linePoints.loc[i,col] == funclinePoints.loc[i,col]:
            continue
        else:
            print(i, col)
            same = False
#     print(col, same)

In [ ]:
axis = [305, 310, 1850, 1950]
plt.subplot(311)
plt.axis(axis)
for col in horCols:
    plt.plot(linePoints.index, linePoints[col])
plt.gca().invert_yaxis()
# plt.subplot(312)
# plt.axis(axis)
for col in horCols:
    plt.plot(linePoints.index, linePoints[col]) 

    
# plt.gca().invert_yaxis()
plt.show()

In [ ]:
i = 0
col = 'watersurface'
# linePoints.loc[i,col]
funclinePoints.loc[i,col]

In [ ]:

oligUC = linePoints['OligUC_z'] != funclinePoints['OligUC_z']
funclinePoints[oligUC]